## Dependencies

---

In [1]:
import yaml
import re
from datetime import datetime
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException


In [4]:
# edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")
edge = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
edge.maximize_window()

[WDM] - Downloading: 100%|██████████| 7.70M/7.70M [00:01<00:00, 5.09MB/s]


### Functions:

In [5]:
def resources(name:str) -> dict[str:str]:
    """Read xpath.yml file that contains xpath's to scrape"""

    with open("xpath.yml", "r") as f:
        content:dict[str:str] = yaml.safe_load(f)
        return content[name]

In [6]:
def close_prompt(driver:object) -> None:
    """Close ad window"""

    button:str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [7]:
def choose_category(category:str, driver:object) -> None:
    """Choose ad category"""

    driver.find_element(by=By.XPATH, value=category).click()

In [8]:
def click_element(element:object, path:str) -> None:
    """Click element on the current document"""

    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [9]:
def extract_description(element:object, path:str) -> str:
    """Extract ad description on the current document"""

    description:str = element.find_element(by=By.XPATH, value=path)
    return description

In [10]:
def extract_feature(element:object, path:str) -> list[str]:
    """Extract some feature from list document based on xpath used."""

    feature = []

    try:
        feature:str = element.find_elements(by=By.XPATH, value=path)
        feature = [data_point.text for data_point in feature]
        return feature
    except Exception:
        feature.append("Web wasn't able to scrape the feature")

In [11]:
def persistence(dataframe:pd.DataFrame) -> None:
    """Generate .csv file from data scraped"""

    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [12]:
def get_len(features:list[list[str]]) -> bool:
    """
    Check if all features has the same length to create new df
    :param features: list of features extracted with scraping
    :return: a list with all lengths where first place is the len of the first element and the mostly accepted.
    """

    base_len = len(features[0])
    lengths = []
    lengths.append(base_len)
    for feature in features:
        if len(feature) == base_len:
            continue
        else:
             lengths.append(len(feature))
    return lengths

In [13]:
def check_len(lengths:list[int]) -> bool:
    """
    Check if all elements in list has the same value.
    :param lengths: list of lengths
    :return: a boolean
    """
    for length in lengths[1:]:
        if length != lengths[0]:
            return False
        return True

Variables pool.

In [14]:
descriptions:list[str] = []
prices:list[int] = []
models:list[str] = []
years:list[int] = []
kilometers:list[int] = []
engines:list[str] = []

# seconds
DELAY:int = 1
SEDAN = resources("cars_sedan")

Get web & close initial prompt.

In [16]:
edge.get(SEDAN["url"])

In [17]:
# close_prompt(edge)

In [18]:
elements = edge.find_elements(by=By.XPATH, value=SEDAN["ad"])

Extract descriptions.

In [19]:
elements[0].click()
try:
    for element in range(len(elements)):
        dynamic_delay = WebDriverWait(edge, DELAY).until(EC.presence_of_element_located((By.XPATH, SEDAN["description"])))
        temp = extract_description(edge, SEDAN["description"])
        descriptions.append(temp.text)
        click_element(edge, SEDAN["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, SEDAN["exit"])

Extract prices & models.

In [20]:
prices = extract_feature(edge, SEDAN["price"])
models = extract_feature(edge, SEDAN["model"])

Extract car year, engine & kilometers.

In [21]:
years = extract_feature(edge, SEDAN["year"])
engines = extract_feature(edge, SEDAN["engine"])
kilometers = extract_feature(edge, SEDAN["kilometers"])

In [22]:
lengths = [prices, models, years, engines, kilometers, descriptions]
get_len(lengths)

[36, 33, 35]

## Export

---

### Functions:

In [23]:
def parse_price(dataframe: pd.DataFrame, feature:str, pattern:str) -> None:
    """
    Parse financial features like prices. Works with $#,### (not dot)
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param pattern: feature pattern to transform
    """
    dataframe[feature] = dataframe [feature] \
        .str.extract(pattern) \
        .applymap(lambda price: int(price.replace(",","")))


In [24]:
def remove_chars(dataframe: pd.DataFrame, feature:str, wildcards:str) -> pd.Series:
    """
    Remove escaped characters from features like indented large extracts of text
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param wildcards: escaped characters to remove
    :return: Serie without escaped characters
    """
    return (dataframe
    .apply(
        lambda row: row[feature]
        ,axis = 1
    )
    .apply(
        lambda extracted_f: re.sub(wildcards, r' ', extracted_f)
    )
    )


In [25]:
PRICE_RG: str = r"^\$(?P<prices>\d+,?\d+)?"
KM_REG: str = r"(?P<km>^\d+)"
CHARACTER_RG:str = '(\n|\r)+'

Remove noise from `model` and `years` lists

In [26]:
years.remove("Ver más")

ValueError: list.remove(x): x not in list

In [27]:
models.remove("Blue Book Encuentra24")

ValueError: list.remove(x): x not in list

In [71]:
lengths = [len(engines), len(models), len(years), len(kilometers), len(prices),len(descriptions)]
lengths

[35, 35, 35, 35, 35, 35]

Create dataframe to transform data extracted

In [72]:
if check_len(lengths):

    df_temp = pd.DataFrame({
        "model": models,
        "year": years,
        "kilometers": kilometers,
        "engine": engines,
        "price": prices,
        "description": descriptions
    })

    # Parse features. String to number and raw string with characters to structured string.
    parse_price(df_temp, "price", PRICE_RG)
    df_temp["kilometers"] = df_temp["kilometers"].str.extract(KM_REG).applymap(int)
    df_temp["year"] = df_temp["year"].map(int)
    df_temp["description"] = remove_chars(df_temp, "description", CHARACTER_RG)

    persistence(df_temp)


In [73]:
df_temp

,model,year,kilometers,engine,price,description
0,Mitsubishi Lancer,2017,68000,Gasolina,8400,Color : Azul Vendo Mitsubishi Lancer 2017 Full...
1,Mitsubishi Mirage G4,2018,60000,Gasolina,8400,Color : Blanco Se vende Mitsubishi G4 2018. Au...
2,Hyundai Accent,2019,78000,Gasolina,9800,Precio negociable: si Color : Negro Hyundai Ac...
3,Hyundai Elantra,2015,98000,Gasolina,9100,Financiamiento: no Precio negociable: si Color...
4,Kia Rio,2019,107000,Gasolina,7699,Precio negociable: si Kia Rio 2019 en camino a...
5,Nissan Sentra,2019,49000,Gasolina,8500,Precio negociable: si WhatsApp 7435- 5520 50 m...
6,Toyota Corolla,2007,128,Gasolina,5500,DESCRIPCIÓN CALIFICACIONES SEGURIDAD Se vende ...
7,Kia Forte,2018,69038,Gasolina,8950,Color : negro Vendo kia forte 2018 motor 2.0 c...
8,Scion xB,2006,220012,Gasolina,4000,Precio negociable: si Color : Ocre vendo scion...
9,Hyundai Accent,2007,155000,Gasolina,4600,Precio negociable: si Color : Gris rata DESCRI...


In [77]:
temp = pd.read_csv("persistence/July-02.csv")
temp

,model,year,kilometers,engine,price,description
0,Mitsubishi Mirage G4,2020,14445,Gasolina,7950,Color : Azul Vendo Mitsubishi Lancer 2017 Full...
1,Mitsubishi Lancer,2017,68000,Gasolina,8400,Color : Blanco Se vende Mitsubishi G4 2018. Au...
2,Mitsubishi Mirage G4,2018,60000,Gasolina,8400,Precio negociable: si Color : AZUL 3 modos de ...
3,Hyundai Elantra,2018,77000,Gasolina,10500,Precio negociable: si Color : Negro Hyundai Ac...
4,Hyundai Accent,2019,78000,Gasolina,9800,Color : negro Vendo kia forte 2018 motor 2.0 c...
5,Kia Forte,2018,69038,Gasolina,8950,Precio negociable: si Color : Ocre vendo scion...
6,Scion xB,2006,220012,Gasolina,4200,Financiamiento: no Precio negociable: si Color...
7,Nissan Sentra,2016,83000,Gasolina,8000,Precio negociable: si Color : Gris rata DESCRI...
8,Hyundai Accent,2007,155000,Gasolina,4600,Color : ROJO DESCRIPCIÓN CALIFICACIONES SEGURI...
9,Hyundai Elantra,2015,150355,Gasolina,7975,Precio negociable: si Color : Cafe DESCRIPCIÓN...
